In [7]:
from __future__ import print_function

import numpy as np
import tensorflow as tf

from keras.utils import pad_sequences, to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.datasets import imdb
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU
from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt

2. Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.

In [8]:
filename = "wonderland.txt"

In [9]:
with open(filename, 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


In [10]:
nb_chars

39

In [11]:
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}


In [12]:
SEQLEN, STEP = 100, 1
input_chars, label_chars = [], []

for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [13]:
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [14]:
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 5 
NUM_EPOCHS_PER_ITERATION = 25
NUM_PREDS_PER_EPOCH = 100

In [ ]:
model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1
        
        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
Epoch 1/25
1095/1095 [==============================] - 67s 43ms/step - loss: 2.2826
Epoch 2/25
1095/1095 [==============================] - 47s 43ms/step - loss: 1.8941
Epoch 3/25
1095/1095 [==============================] - 52s 47ms/step - loss: 1.7462
Epoch 4/25
1095/1095 [==============================] - 48s 44ms/step - loss: 1.6417
Epoch 5/25
1095/1095 [==============================] - 48s 43ms/step - loss: 1.5609
Epoch 6/25
1095/1095 [==============================] - 47s 43ms/step - loss: 1.4970
Epoch 7/25
1095/1095 [==============================] - 47s 43ms/step - loss: 1.4443
Epoch 8/25
1095/1095 [==============================] - 50s 45ms/step - loss: 1.4002
Epoch 9/25
1095/1095 [==============================] - 47s 43ms/step - loss: 1.3628
Epoch 10/25
1095/1095 [==============================] - 46s 42ms/step - loss: 1.3292
Epoch 11/25
1095/1095 [==============================] - 46s 42ms/step - loss: 1.2997
Epoch 12/25
1095/1095 [==========================

In [15]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
sentences = sent_tokenize(text)
len(max(sentences, key=len))

919

In [17]:
sum( map(len, sentences) ) / len(sentences)


84.05507955936352

In [26]:
sentences_list = list(map(list, sentences))

In [29]:
for sent in sentences_list:
    for i, ch in enumerate(sent):
        sent[i] = char2index[ch]
        

In [48]:
SENTLEN, SEQLEN = 100, 10
X_padded = []
Y_padded = []
for sent in sentences_list:
    for i in range(len(sent)-SEQLEN-1):
        x = sent[i:(i+SEQLEN)]
        #x = [0] * (SEQLEN - len(x)) + x
        X_padded.append(x)
        Y_padded.append(sent[i+SEQLEN])
        #print(x)

X = np.zeros((len(X_padded), SEQLEN, nb_chars), dtype='float32')
y = np.zeros((len(X_padded), nb_chars), dtype='float32')

for i, input_char in enumerate(X_padded):
    for j, ch in enumerate(input_char):
        X[i, j, ch] = 1
    y[i, Y_padded[i]] = 1

In [60]:
model_1 = Sequential()
model_1.add(LSTM(HIDDEN_SIZE))
model_1.add(Dense(nb_chars))
model_1.add(Activation("softmax"))
model_1.compile(loss="categorical_crossentropy", optimizer="adamax")

model_1.fit(X, y, batch_size=32, epochs=150)

Epoch 1/150
3741/3741 [==============================] - 22s 5ms/step - loss: 2.4931
Epoch 2/150
3741/3741 [==============================] - 20s 5ms/step - loss: 2.1183
Epoch 3/150
3741/3741 [==============================] - 20s 5ms/step - loss: 1.9797
Epoch 4/150
3741/3741 [==============================] - 20s 5ms/step - loss: 1.8806
Epoch 5/150
3741/3741 [==============================] - 20s 5ms/step - loss: 1.8052
Epoch 6/150
3741/3741 [==============================] - 20s 5ms/step - loss: 1.7438
Epoch 7/150
3741/3741 [==============================] - 20s 5ms/step - loss: 1.6913
Epoch 8/150
3741/3741 [==============================] - 20s 5ms/step - loss: 1.6449
Epoch 9/150
3741/3741 [==============================] - 20s 5ms/step - loss: 1.6043
Epoch 10/150
3741/3741 [==============================] - 20s 5ms/step - loss: 1.5681
Epoch 11/150
3741/3741 [==============================] - 20s 5ms/step - loss: 1.5339
Epoch 12/150
3741/3741 [==============================] - 20s 5

In [68]:
for i in range(10):

    test_idx = np.random.randint(len(X_padded))
    test_chars = X_padded[test_idx]

    test_str = ""
    for i in test_chars:
        test_str = test_str + index2char[i]

    print(f"Seed: {test_str}")

    for i in range(100):

        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, ch] = 1

        pred = model_1.predict(X_test, verbose=0)[0]
        ch_idx = np.argmax(pred)
        y_pred = index2char[ch_idx]

        print(y_pred, end="")

        test_chars = test_chars[1:]
        test_chars.append(ch_idx)
    print('\n')

Seed: d felt qui
te strange at rablingly ready to ask help to stime, very nearly carried it out into the wood seme, t

Seed: ther late,
 and the poor little thing sat down a good in into uped down the chimney this bottle doest to twongh

Seed:  mock turt
le sighed deeply, and the poor little thing sat down a good in into uped down the chimney this bottl

Seed: their putt
ing their heads down, as she remembered the words did not seem to draw herehthe heads of the court, 

Seed:  prove i d
idnt know the way of expecting to see it to her feet as the stightyed turter in a very heary off her

Seed: ent on and
 how do the may meat how ravent alice felt a very foor a gount haig to herself, as it we king of a v

Seed: way wherev
er you go downing to ind the sobs, with a subjer of shaking the paisious shus in a longurious sting,

Seed: the house 
of the mallets live flamingo was gone in a moment to be two peopless so every lone as it this wither

Seed:  like the 
look of the soun, and the poor 

1. Для обучения использовался полный текст "Алисы в стране чудес".

2. Модель обучалась на отрывках проедложений, а не на соучайных помледовательностях из текста.

3. Использована модель LSTM и оптимизатор Adamax.

4. Уменьшен размер батча и увеличено количество эпох.

При доступе к более мощным процессорам GPU можно попробовать изменить следующие параметры:

* Сократить batch size до минимума.
* Увеличить количесво эпох до нескольких сотен.
* Попробовать другое разбиение текста (на более крупные отрезки).
* Увеличить количество слоев модели.